In [105]:
import numpy as np
from dppy.finite_dpps import FiniteDPP

r, N = 4, 10 #r number of features and N number of videos

#L-Ensemble of rank r that is not a projection DPP
Phi = np.random.randn(r, N)
DPP_L = FiniteDPP('likelihood', **{'L_gram_factor': Phi})

#Projection DPP with the corresponding previous eigenvectors
L = np.matmul(np.transpose(Phi), Phi)
e_vals, e_vecs = np.linalg.eigh(L)
proj_DPP = FiniteDPP('correlation', projection=True,
                    **{'K_eig_dec': (np.concatenate((np.zeros(N-r),np.ones(r))), e_vecs)})

L_dual = Phi Phi.T was computed: Phi (dxN) with d<N


In [50]:
for _ in range(10):
    print(DPP_L.sample_exact())

[5, 9, 6]
[0, 9, 1, 4]
[9, 4]
[9, 1, 3, 6]
[6, 1, 0]
[5, 6, 4]
[8, 3, 0]
[5, 0, 2]
[1, 9, 4]
[4, 9, 1]


In [81]:
for _ in range(10):
    print(proj_DPP.sample_exact())

[5, 6, 0, 1]
[7, 2, 5, 1]
[0, 6, 7, 3]
[3, 4, 7, 0]
[8, 2, 3, 7]
[1, 7, 3, 0]
[2, 1, 0, 6]
[6, 0, 3, 4]
[1, 0, 2, 6]
[1, 6, 2, 0]


In [119]:
#Compute singleton marginal of r-DPP using formula (205) of 'DPP for Machine Learning'
from dppy.exact_sampling import elementary_symmetric_polynomials
e_N_r = elementary_symmetric_polynomials(e_vals, r)[r, N]
e_minus_n_r_minus_one = np.array(np.zeros(N))

for n in range(N):
    e_minus_n_r_minus_one[n] = elementary_symmetric_polynomials(np.concatenate((e_vals[:n], e_vals[n+1:])), r-1)[r-1, N-1]

singleton_marginals = np.array(np.zeros(N))
for i in range(N):
    for n in range(N):
        singleton_marginals[i] += e_vecs[i,n]**2*e_vals[n]*e_minus_n_r_minus_one[n]/e_N_r

print("Singleton marginals of r-DPP")
print(singleton_marginals)
print('\n')

print("Singleton marginals of associated projection DPP")
proj_DPP.compute_K()
print([proj_DPP.K[i,i] for i in range(N)])
print('\n')

print('Singleton marginals of L-Ensemble')
DPP_L.compute_K()
print([DPP_L.K[i,i] for i in range(N)])
print('\n')

Singleton marginals of r-DPP
[0.74472789 0.6079976  0.28690022 0.42289518 0.31740471 0.21377312
 0.47396621 0.26961754 0.27861403 0.38410349]


Singleton marginals of associated projection DPP
[0.744727894017959, 0.6079975966310457, 0.2869002169268609, 0.42289518443439356, 0.31740470665803167, 0.2137731170898463, 0.47396621345955525, 0.2696175422136192, 0.27861403422870407, 0.3841034943399867]


Singleton marginals of L-Ensemble
[0.6821750312785398, 0.5774367606066863, 0.2471161199490018, 0.35534135509732373, 0.29500844137922283, 0.18943830447993093, 0.4285562710961981, 0.242537082556097, 0.2600229000260916, 0.36029036757896915]


